## Generate more WR sample data

In [6]:
!pip3 install pandas sklearn
!pip3 install matplotlib

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/opt/homebrew/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [26]:
import json
import pandas as pd
import re
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

In [8]:
import numpy as np

# Needed for plotting
import matplotlib.colors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Needed for generating classification, regression and clustering datasets
import sklearn.datasets as dt

# Needed for generating data from an existing dataset
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

# Import College and Pro Datasets

In [9]:
skill_players = pd.read_csv('../ProData/csvs/skill_player_stats.csv', header=0)
college_wr_data = pd.read_csv('../CollegeData/all_receiving.csv')

# Aggregate Pro Data and Standardize

In [10]:
drop_columns = ['Unnamed: 0', 'team', 'av']

In [11]:
skills_df = skill_players.drop(drop_columns, axis=1)

In [12]:
# Add column with value 1 so that when we group by player 
# and sum we get a count of the number of seasons
skills_df['season_count'] = 1

## Fill null with the player's average

In [13]:
fillna_cols = ['snap_played_percentage', 'snaps_played', 'total_snaps']

def fill_with_player_averages(group):
    group[fillna_cols] = group[fillna_cols].fillna(group[fillna_cols].mean())
    return group

In [14]:
new = skills_df.groupby('player_id', as_index=False).apply(fill_with_player_averages)

## For any left over nulls, fill with position average for games started

In [15]:
new[fillna_cols] = new[fillna_cols].fillna(new[fillna_cols].mean())

In [16]:
new[fillna_cols] = new[fillna_cols].replace(0, new[fillna_cols].mean())

In [17]:
new.dropna(inplace=True)

## Map player_id to position
After grouping and aggregating data, non-numeric features will be removed. By saving the position and player ID, we can add the position back to the player's data after aggregation.

In [18]:
player_to_position = new[['player_id', 'position']].drop_duplicates().set_index('player_id')

# Aggregate career statistics for each player
Roll up every season each player playerd into one data point using averages and percentages as opposed to career totals.

In [19]:
career_stats = new.groupby('player_id').sum()

In [20]:
fields_to_divide_by_season = [
    'rec_yds_per_rec', 'catch_pct', 'rec_yds_per_tgt',
    'rush_yds_per_att', 'rush_att_per_g', 'yds_per_touch'
]

fields_to_divide_by_season.extend([col for col in career_stats.columns if col.endswith('per_g')])

In [21]:
for field in fields_to_divide_by_season:
    career_stats[field] = career_stats[field] / career_stats.season_count

In [22]:
per_game_avg_fields = [
    ('targets_per_g', 'targets'),
    ('touches_per_g', 'touches'),
    ('yds_from_scrimmage_per_g', 'yds_from_scrimmage'),
    ('rush_receive_td_per_g', 'rush_receive_td'),
    ('rec_td_per_g', 'rec_td'),
    ('rush_first_down_per_g', 'rush_first_down'),
    ('fumbles_per_g', 'fumbles'),
    ('rush_td_per_g', 'rush_td'),
    ('rec_first_down_per_g', 'rec_first_down'),
]

In [23]:
for (new_field, metric) in per_game_avg_fields:
    career_stats[new_field] = career_stats[metric] / career_stats.g

In [24]:
career_stats['snap_percentage'] = career_stats.snaps_played / career_stats.total_snaps

In [75]:
def get_fields_to_remove(career_stats):
    per_game_fields = [col for col in career_stats.columns if col.endswith('per_g')]
    not_per_game_fields = [col for col in career_stats.columns if not col.endswith('per_g')]

    fields_to_remove = []

    for field in not_per_game_fields:
        for field_2 in per_game_fields:
            base_field = re.match('.*(?=_per_g)', field_2)
            if base_field[0] == field:
                fields_to_remove.append(field)
                continue
    return fields_to_remove

pro_fields_to_remove = get_fields_to_remove(career_stats)
pro_fields_to_remove.extend([
    'age', 'rec_long', 'rush_long', 'total_snaps', 
    'snap_played_percentage','g', 'gs', 'snaps_played'
])

In [28]:
career_stats = career_stats.drop(pro_fields_to_remove, axis=1)

In [29]:
career_stats.query('player_id == "AdamDa01"')

,rec_yds_per_rec,rec_per_g,rec_yds_per_g,catch_pct,rec_yds_per_tgt,rush_yds_per_att,rush_yds_per_g,rush_att_per_g,yds_per_touch,season_count,targets_per_g,touches_per_g,yds_from_scrimmage_per_g,rush_receive_td_per_g,rec_td_per_g,rush_first_down_per_g,fumbles_per_g,rush_td_per_g,rec_first_down_per_g,snap_percentage
player_id,,,,,,,,,,,,,,,,,,,,
AdamDa01,12.0875,5.75,70.3125,63.875,7.75,0.0,0.0,0.0,12.0875,8,8.611111,5.592593,68.092593,0.601852,0.601852,0.0,0.046296,0.0,3.37963,0.832297


## Add position back to dataset

In [30]:
career_stats_with_position = career_stats.join(player_to_position)

## Extract WR Data

In [33]:
wr_data = career_stats_with_position[career_stats_with_position.position == 'WR']

# Aggregate College Dataset

In [79]:
college_wr_data['season_count'] = 1
grouped_data = college_wr_data.groupby('playerId').sum()
college_feature_data = grouped_data
for feature in grouped_data:
    if feature == 'g':
        college_feature_data['g'] = grouped_data['g']
        continue
    elif '_per_' in feature:
        college_feature_data[feature] = college_feature_data[feature] / college_feature_data.season_count
    else:
        college_feature_data[f'{feature}_per_g'] = grouped_data[feature] / grouped_data['g']

In [83]:
college_fields_to_remove = get_fields_to_remove(college_feature_data)
college_fields_to_remove.append('season_count_per_g')
college_feature_data.drop(college_fields_to_remove, axis=1, inplace=True)

In [84]:
college_feature_data.columns

Index(['g', 'rec_yds_per_rec', 'rush_yds_per_att', 'scrim_yds_per_att',
       'rec_per_g', 'rec_yds_per_g', 'rec_td_per_g', 'rush_att_per_g',
       'rush_yds_per_g', 'rush_td_per_g', 'scrim_att_per_g', 'scrim_yds_per_g',
       'scrim_td_per_g'],
      dtype='object')

# Add CollegeId to professional dataset and then inner join

In [38]:
match_data = {}
with open('./NFL EDA/matches.json', 'r') as match_file:
  match_data = json.load(match_file)

In [39]:
player_id_tuple = [(match['nfl_id'], match['college_id']) for match in match_data]

nfl_id, college_id = list(zip(*player_id_tuple))
player_ids = pd.DataFrame.from_dict({
  'nfl_id': nfl_id,
  'college_id': college_id
}).set_index('nfl_id')

In [116]:
pro_data_with_college_id = wr_data.join(player_ids, how='inner')

In [110]:
pro_data_with_college_id.to_csv('./Pro_WR_Career_Data_with_College_Id.csv')

In [117]:
pro_data_with_college_id.reset_index(inplace=True)
pro_data_with_college_id = pro_data_with_college_id.add_suffix('_pro')
pro_data_with_college_id['college_id'] = pro_data_with_college_id['college_id_pro']
pro_data_with_college_id.drop(['college_id_pro', 'index_pro'], axis=1, inplace=True)

In [118]:
college_feature_data.reset_index(inplace=True)

In [119]:
combined_data = pro_data_with_college_id.merge(
    college_feature_data,
    left_on='college_id',
    right_on='playerId',
    suffixes=['_pro', '_college'],
    how='inner'
)

In [120]:
combined_data = combined_data.drop('index', axis=1)

In [121]:
len(combined_data)

455

In [122]:
combined_data.to_csv('./College_And_Pro_WR_CareerData.csv')

# Create Synthetic Data

In [61]:
import numpy as np

# # Needed for plotting
# import matplotlib.colors
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# # Needed for generating classification, regression and clustering datasets
# import sklearn.datasets as dt

# Needed for generating data from an existing dataset
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

In [94]:
# Define the seed so that results can be reproduced
seed = 11
rand_state = 11

In [124]:
train_data = combined_data.drop(['college_id', 'playerId', 'position_pro'], axis=1)

In [125]:
# Fit a kernel density model using GridSearchCV to determine the best parameter for bandwidth
bandwidth_params = {'bandwidth': np.arange(0.01,1,0.05)}
grid_search = GridSearchCV(KernelDensity(), bandwidth_params)
grid_search.fit(train_data)
kde = grid_search.best_estimator_

# Generate/sample 8 new faces from this dataset
new_data = kde.sample(2000, random_state=rand_state)
synthetic_data = pd.DataFrame.from_records(new_data, columns=train_data.columns)


In [126]:
synthetic_data

,rec_yds_per_rec_pro,rec_per_g_pro,rec_yds_per_g_pro,catch_pct_pro,rec_yds_per_tgt_pro,rush_yds_per_att_pro,rush_yds_per_g_pro,rush_att_per_g_pro,yds_per_touch_pro,season_count_pro,...,scrim_yds_per_att,rec_per_g,rec_yds_per_g,rec_td_per_g,rush_att_per_g,rush_yds_per_g,rush_td_per_g,scrim_att_per_g,scrim_yds_per_g,scrim_td_per_g
0,12.920178,2.506718,30.102957,49.719250,6.097599,2.516050,1.702839,-1.161315,13.503552,4.561261,...,13.594572,5.365196,88.521555,0.411062,2.336268,5.499654,-0.534142,6.263915,94.216866,-0.508675
1,13.631427,1.027938,14.404005,67.128489,8.866538,-0.320312,-1.147866,0.219630,14.021723,4.634700,...,15.968334,4.483120,80.355576,-0.160566,-0.104481,0.551911,-0.900498,5.226173,79.968721,0.241271
2,13.043600,7.166934,83.339870,69.020924,10.088185,5.971695,-0.813025,0.046359,14.938563,2.997772,...,11.362909,5.707668,80.067826,1.690494,0.808875,0.304265,0.781420,5.139380,81.647464,0.770418
3,13.475311,1.534359,7.327607,67.998193,7.361610,1.272094,0.620708,-1.358463,13.926447,0.777793,...,9.736818,2.378874,22.689172,-3.281119,1.023816,5.567331,1.135430,3.102424,30.466355,0.987394
4,16.449291,3.632731,69.783382,60.629014,9.871173,1.121441,2.447580,0.773306,16.414860,9.838162,...,13.707873,4.157653,71.996765,0.406409,0.914378,8.171408,-1.541317,4.911633,78.636446,0.250861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,15.827375,0.981684,33.342295,63.452903,11.565156,-1.916145,0.613330,1.680740,16.034245,3.045555,...,14.364315,2.836295,50.787155,0.354783,-0.376047,0.978344,0.828132,4.188590,49.828517,1.412072
1996,8.354010,2.820466,22.487967,49.109317,3.945074,0.934562,1.282902,0.166285,8.301482,6.197101,...,17.017755,3.927428,77.483794,0.610065,-0.330179,0.695940,-0.811871,4.845797,77.714471,1.016284
1997,8.321823,2.050399,33.939478,32.779032,6.290951,0.635451,-0.310918,-0.022197,7.539523,4.021361,...,12.611050,2.628443,81.454673,0.566019,-0.692465,2.366390,-1.645301,3.549630,85.612856,0.440606
1998,15.762549,1.556822,35.566673,45.156703,6.644437,-2.820178,-0.012299,0.763661,16.403174,3.018775,...,16.215372,1.918688,51.550231,2.019247,-2.132208,0.627170,-0.843708,3.306177,50.395153,-1.510871


# Score and Tier the Synthetic Data

In [128]:
from sklearn.decomposition import PCA

In [167]:
pro_features = [feature for feature in synthetic_data.columns if feature.endswith('_pro')]
pro_features.remove('season_count_pro')

In [142]:
pca = PCA(n_components=1)
pca_analysis = pd.DataFrame(
    pca.fit_transform(synthetic_data[pro_features]), 
    columns=['PCA%i' % i for i in range(1)], 
    index=synthetic_data.index
)

In [143]:
synthetic_with_scores = synthetic_data.merge(pca_analysis, left_index=True, right_index=True)
synthetic_with_scores['pro_score'] = synthetic_with_scores.PCA0
synthetic_with_scores.drop('PCA0', axis=1, inplace=True)

In [144]:
synthetic_with_scores.to_csv('./Synthetic_data_with_scores')

In [145]:
synthetic_with_scores.columns

Index(['rec_yds_per_rec_pro', 'rec_per_g_pro', 'rec_yds_per_g_pro',
       'catch_pct_pro', 'rec_yds_per_tgt_pro', 'rush_yds_per_att_pro',
       'rush_yds_per_g_pro', 'rush_att_per_g_pro', 'yds_per_touch_pro',
       'season_count_pro', 'targets_per_g_pro', 'touches_per_g_pro',
       'yds_from_scrimmage_per_g_pro', 'rush_receive_td_per_g_pro',
       'rec_td_per_g_pro', 'rush_first_down_per_g_pro', 'fumbles_per_g_pro',
       'rush_td_per_g_pro', 'rec_first_down_per_g_pro', 'snap_percentage_pro',
       'g', 'rec_yds_per_rec', 'rush_yds_per_att', 'scrim_yds_per_att',
       'rec_per_g', 'rec_yds_per_g', 'rec_td_per_g', 'rush_att_per_g',
       'rush_yds_per_g', 'rush_td_per_g', 'scrim_att_per_g', 'scrim_yds_per_g',
       'scrim_td_per_g', 'pro_score'],
      dtype='object')

# Train Model on Synthetic Data

## Tier the Synthetic Data

In [190]:
TIERS = 5

sorted_scores = synthetic_with_scores.sort_values(by='pro_score').reset_index()
number_of_players = len(sorted_scores)
increment = number_of_players // TIERS

sorted_scores['tier'] = -1
prev_idx = 0
for tier_num, i in enumerate(range(0, number_of_players, increment)):
    print(tier_num)
    increment_end = prev_idx + increment
    print(f'getting players {prev_idx} - {increment_end}')
    if tier_num + 1 == TIERS:
        sorted_scores.loc[prev_idx:, 'tier'] = tier_num
    else:
        sorted_scores.loc[prev_idx:increment_end, 'tier'] = tier_num

    prev_idx = i + increment
  
target = sorted_scores

0
getting players 0 - 400
1
getting players 400 - 800
2
getting players 800 - 1200
3
getting players 1200 - 1600
4
getting players 1600 - 2000


In [156]:
target.to_csv('./TieredSynthetic_WR_Data.csv')

## Train Synthetic Data

In [193]:
training_features = [feat for feat in target.columns if not feat.endswith('_pro') and feat not in ['tier', 'index', 'pro_score']]

In [161]:
synthetic_features = target[training_features]
synthetic_target = target.tier

In [162]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 6).fit(synthetic_features, synthetic_target)

# Test Model with real data

## Score and Tier Natural Data

In [163]:
combined_data.columns

Index(['rec_yds_per_rec_pro', 'rec_per_g_pro', 'rec_yds_per_g_pro',
       'catch_pct_pro', 'rec_yds_per_tgt_pro', 'rush_yds_per_att_pro',
       'rush_yds_per_g_pro', 'rush_att_per_g_pro', 'yds_per_touch_pro',
       'season_count_pro', 'targets_per_g_pro', 'touches_per_g_pro',
       'yds_from_scrimmage_per_g_pro', 'rush_receive_td_per_g_pro',
       'rec_td_per_g_pro', 'rush_first_down_per_g_pro', 'fumbles_per_g_pro',
       'rush_td_per_g_pro', 'rec_first_down_per_g_pro', 'snap_percentage_pro',
       'position_pro', 'college_id', 'playerId', 'g', 'rec_yds_per_rec',
       'rush_yds_per_att', 'scrim_yds_per_att', 'rec_per_g', 'rec_yds_per_g',
       'rec_td_per_g', 'rush_att_per_g', 'rush_yds_per_g', 'rush_td_per_g',
       'scrim_att_per_g', 'scrim_yds_per_g', 'scrim_td_per_g'],
      dtype='object')

In [170]:
pro_features = [feature for feature in synthetic_data.columns if feature.endswith('_pro')]
pro_features.remove('season_count_pro')

In [171]:
pca = PCA(n_components=1)
pca_analysis = pd.DataFrame(
    pca.fit_transform(combined_data[pro_features]), 
    columns=['PCA%i' % i for i in range(1)], 
    index=combined_data.index
)

In [172]:
combined_data_with_scores = combined_data.merge(pca_analysis, left_index=True, right_index=True)
combined_data_with_scores['pro_score'] = combined_data_with_scores.PCA0
combined_data_with_scores.drop('PCA0', axis=1, inplace=True)

In [173]:
combined_data_with_scores.to_csv('./Synthetic_data_with_scores')

### Tier the players based on score

In [194]:
TIERS = 5

sorted_scores = combined_data_with_scores.sort_values(by='pro_score').reset_index()
number_of_players = len(sorted_scores)
increment = number_of_players // TIERS

sorted_scores['tier'] = -1
prev_idx = 0
for tier_num, i in enumerate(range(0, number_of_players, increment)):
    print(tier_num)
    increment_end = prev_idx + increment
    print(f'getting players {prev_idx} - {increment_end}')
    if tier_num + 1 == TIERS:
        sorted_scores.loc[prev_idx:, 'tier'] = tier_num
    else:
        sorted_scores.loc[prev_idx:increment_end, 'tier'] = tier_num

    prev_idx = i + increment
  
target = sorted_scores

0
getting players 0 - 91
1
getting players 91 - 182
2
getting players 182 - 273
3
getting players 273 - 364
4
getting players 364 - 455


In [175]:
target.to_csv('./Tiered_WR_Data.csv')

## Test Model created from synthetic data

In [180]:
target.columns

Index(['index', 'rec_yds_per_rec_pro', 'rec_per_g_pro', 'rec_yds_per_g_pro',
       'catch_pct_pro', 'rec_yds_per_tgt_pro', 'rush_yds_per_att_pro',
       'rush_yds_per_g_pro', 'rush_att_per_g_pro', 'yds_per_touch_pro',
       'season_count_pro', 'targets_per_g_pro', 'touches_per_g_pro',
       'yds_from_scrimmage_per_g_pro', 'rush_receive_td_per_g_pro',
       'rec_td_per_g_pro', 'rush_first_down_per_g_pro', 'fumbles_per_g_pro',
       'rush_td_per_g_pro', 'rec_first_down_per_g_pro', 'snap_percentage_pro',
       'position_pro', 'college_id', 'playerId', 'g', 'rec_yds_per_rec',
       'rush_yds_per_att', 'scrim_yds_per_att', 'rec_per_g', 'rec_yds_per_g',
       'rec_td_per_g', 'rush_att_per_g', 'rush_yds_per_g', 'rush_td_per_g',
       'scrim_att_per_g', 'scrim_yds_per_g', 'scrim_td_per_g', 'pro_score',
       'tier'],
      dtype='object')

In [195]:
excludes = ['index', 'college_id', 'position_pro', 'playerId', 'pro_score', 'tier']
training_features = [feat for feat in target.columns if not feat.endswith('_pro') and feat not in excludes]
len(training_features)

13

In [196]:
# accuracy on X_test
test_features = target[training_features]
test_target = target.tier
accuracy = knn.score(test_features, test_target)
print(accuracy)
 
# # creating a confusion matrix
# knn_predictions = knn.predict(X_test)
# cm = confusion_matrix(y_test, knn_predictions)

0.8065934065934066


In [197]:
len(target)

455

# Determine Feature Importance

In [301]:
knn_predictions = knn.predict(test_features)
cm = confusion_matrix(test_target, knn_predictions)
print(cm)

[[80  7  1  0  3]
 [ 9 74  3  4  1]
 [ 7  7 65  8  4]
 [ 5  5  2 74  5]
 [ 2  5  5  5 74]]


# Determine average draft position of each tier

In [212]:
with open('../ProData/cleaned/draft_data/WR.json', 'r') as infile:
    draft_data = json.load(infile)

In [204]:
tiers_with_nfl_id = target[['tier', 'college_id']].merge(player_ids.reset_index(), left_on='college_id', right_on='college_id')

tiers_with_nfl_id = tiers_with_nfl_id.drop('college_id', axis=1)

In [206]:
tier_groups = tiers_with_nfl_id.groupby('tier')

In [219]:
tier_to_draft_position_array = {}

for tier, data in tier_groups:
    draft_positions = [draft_data[player]['draft_pick'] for player in data.nfl_id]
    tier_to_draft_position_array[tier] = draft_positions

In [220]:
avg_draft_position_per_tier = {tier: np.mean(np.array(draft_positions)) 
                               for tier, draft_positions in tier_to_draft_position_array.items()}

In [221]:
avg_draft_position_per_tier

{0: 155.96703296703296,
 1: 136.87912087912088,
 2: 118.23076923076923,
 3: 80.46153846153847,
 4: 57.81318681318681}

# Determine how accurate NFL is at drafting players into correct tier
Given all WRs drafted in any draft, how often do NFL teams pick the player in the correct tier

In [302]:
# Find superset of draft positions of all NFL WRs from last 20 years
all_draft_positions = sorted([draft_data[player]['draft_pick'] for player in tiers_with_nfl_id.nfl_id])
tier_size = len(all_draft_positions) // TIERS

tier_draft_range = []
i = 0
while i < len(all_draft_positions):
    tier_start = all_draft_positions[i]
    i += tier_size
    
    try:
        tier_end = all_draft_positions[i]
    except IndexError:
        print('last group')
        tier_end = all_draft_positions[-1]
        tier_draft_range.append((tier_start, tier_end))
        break
    
    # Check if next position is equal to current position
    # then advance iterator until there is a change
    if all_draft_positions[i+1] == tier_end:
        i += 1
        while all_draft_positions[i] == tier_end:
            i += 1
    
    i += 1
    tier_draft_range.append((tier_start, tier_end))
    
tier_draft_range = sorted(tier_draft_range, key=lambda x: -x[0])
tier_draft_range

last group


[(199, 256), (127, 197), (80, 126), (40, 79), (2, 39)]

In [288]:
# tier_draft_range = [(205, 300), (154, 204), (103, 153), (52, 102), (1, 51)]

# returns 1 if player drafted in expected range, otherwise 0
def find_clostest_tier_given_draft_position(tier, draft_position):
    expected_draft_start, expected_draft_end = tier_draft_range[tier]
    if draft_position >= expected_draft_start and draft_position <= expected_draft_end:
        return 1
    else:
        return 0

In [289]:
tier_and_player_id_tuples = tiers_with_nfl_id[['tier', 'nfl_id']].set_index('tier').to_records()
draft_hits = []

for tier, player_id in tier_and_player_id_tuples:
    val = find_clostest_tier_given_draft_position(tier, draft_data[player_id]['draft_pick'])
    draft_hits.append(val)
    
np.mean(np.array(draft_hits))

0.3032967032967033

# Draft accuracy by tier

In [290]:
from collections import defaultdict

In [291]:
tier_and_player_id_tuples = tiers_with_nfl_id[['tier', 'nfl_id']].set_index('tier').to_records()
draft_hits = defaultdict(list)

for tier, player_id in tier_and_player_id_tuples:
    print(tier, player_id, draft_data[player_id]['draft_pick'])
    val = find_clostest_tier_given_draft_position(tier, draft_data[player_id]['draft_pick'])
    draft_hits[tier].append(val)
    
for tier, hits in draft_hits.items():
    print(tier, np.mean(hits))

0 MaylVi00 123
0 ReedJo03 151
0 GandAn00 142
0 KentJo20 210
0 HallRo02 169
0 PaytJo00 154
0 PondWi00 199
0 JohnMa04 229
0 GilyMa00 99
0 WillPa00 80
0 DaviGe00 186
0 DaviRo00 209
0 WatkTo20 218
0 PilaKe00 132
0 SaunJa00 104
0 SwaiBr00 217
0 ReedDa00 156
0 WinfJu00 187
0 MitcCa00 177
0 MaloJo00 128
0 BakeDa01 227
0 ChesJe00 139
0 DorsKe01 224
0 GreeSk20 125
0 ClevTy00 252
0 MaysLe00 202
0 JoneDa02 226
0 SampDe00 249
0 WyliDe00 107
0 EberJe00 235
0 SweeLi00 53
0 KaspKe00 190
0 TurnPa00 87
0 RobiRy00 118
0 ClayKa00 184
0 WhitJo02 244
0 StreDe00 146
0 HuntJa00 206
0 HowrKe00 221
0 PatmDe00 212
0 MoraJo00 134
0 YounWa00 226
0 JohnRo03 123
0 WhalRy00 167
0 MoorJM00 133
0 DickDo00 227
0 MadiAd00 158
0 RussCl00 87
0 MitcMa00 243
0 GibsSh00 166
0 RobyCo00 68
0 HighJo00 168
0 HemiJu00 238
0 GardTa00 231
0 WillDe05 82
0 ArceJJ00 57
0 ColeCh00 70
0 TaylJa00 168
0 FranWi00 105
0 BoycJo00 102
0 HubbPa00 191
0 JacoTa00 44
0 AusbDa00 241
0 RidgDa00 192
0 DarbAm00 106
0 StewAr00 79
0 WilsTr00 78
0 HayeW

# Draft Accuracy by Confusion Matrix

In [324]:
def find_tier_given_draft_position(tier, draft_position):
    for i, (expected_start, expected_end) in enumerate(tier_draft_range):
        if draft_position >= expected_start and draft_position <= expected_end:
            return i
    return 0

In [325]:
tier_and_player_id_tuples = tiers_with_nfl_id[['tier', 'nfl_id']].set_index('tier').to_records()
draft_hits = defaultdict(list)

for tier, player_id in tier_and_player_id_tuples:
    val = find_tier_given_draft_position(tier, draft_data[player_id]['draft_pick'])
    draft_hits[tier].append(val)

In [326]:
from collections import Counter, OrderedDict

In [327]:
confusion_mat = {}
for tier, draft_tier_list in draft_hits.items():
    draft_tiers = Counter(draft_tier_list).items()
    print(draft_tiers)
    draft_tiers = sorted(draft_tiers)
    print(draft_tiers)
    
    confusion_mat[tier] = [values for _, values in draft_tiers]
    
confusion_mat

dict_items([(2, 20), (1, 32), (0, 29), (3, 9), (4, 1)])
[(0, 29), (1, 32), (2, 20), (3, 9), (4, 1)]
dict_items([(4, 8), (1, 26), (2, 20), (0, 21), (3, 16)])
[(0, 21), (1, 26), (2, 20), (3, 16), (4, 8)]
dict_items([(1, 15), (2, 25), (0, 20), (3, 14), (4, 17)])
[(0, 20), (1, 15), (2, 25), (3, 14), (4, 17)]
dict_items([(2, 16), (4, 28), (1, 15), (3, 27), (0, 5)])
[(0, 5), (1, 15), (2, 16), (3, 27), (4, 28)]
dict_items([(3, 30), (1, 6), (2, 13), (0, 3), (4, 39)])
[(0, 3), (1, 6), (2, 13), (3, 30), (4, 39)]


{0: [29, 32, 20, 9, 1],
 1: [21, 26, 20, 16, 8],
 2: [20, 15, 25, 14, 17],
 3: [5, 15, 16, 27, 28],
 4: [3, 6, 13, 30, 39]}